In [1]:
# from pylabrobot.resources import (
#     Resource,
#     Carrier, MFXCarrier, PlateCarrier, TipCarrier, TroughCarrier, TubeCarrier,
#     Container, PetriDish, Trough, Tube, Well,
#     Deck, OTDeck, HamiltonDeck, TecanDeck,
#     ItemizedResource, Plate, TipRack, TubeRack,
#     ResourceHolder, PlateHolder,
#     Lid,
#     PlateAdapter,
#     ResourceStack
# )
# # TODO: NestedTipRackStack,
# TODO: Tip,
# TODO: Workcell


In [6]:
import ast
import os
import re

def find_python_dirs(directory, ignore_list=None):
    """
    Recursively finds all directories containing .py files within a given directory,
    while ignoring specific filenames provided in ignore_list.
    
    :param directory: The root directory to start searching from.
    :param ignore_list: A list of filenames (without .py extension) to ignore.
    :return: A list of directories containing .py files.
    """
    if ignore_list is None:
        ignore_list = set()
    else:
        ignore_list = set(ignore_list)  # Convert to set for faster lookups
    
    python_dirs = set()
    
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".py"):
                if all([
                    file[:-3] not in ignore_list,
                    "test" not in file,
                    "opentrons" not in root
                ]):
                    python_dirs.add(os.path.join(root, file))
    
    return sorted(python_dirs)

import ast
import os
import re

def extract_functions_with_imports(file_path, class_list, optional_attr=None):
    """
    Parses a Python file to find functions that return instances of specific classes.
    
    :param file_path: Path to the Python file to analyze.
    :param class_list: List of class names to check against.
    :param optional_attr: List of optional attributes to extract from the docstring.
    :return: Dictionary mapping function names to the class they return, including the 
             directory path, catalog numbers, material, manufacturer, manufacturer_link, 
             notes, and additional optional attributes.
    """
    if optional_attr is None:
        optional_attr = []
    
    with open(file_path, "r", encoding="utf-8") as file:
        tree = ast.parse(file.read(), filename=file_path)
    
    imported_classes = set(class_list)  # Use the provided list of classes
    
    function_map = []
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            func_data = {
                "plr_id": node.name,
                "resource_type": None,
                "manufacturer": None,
                "brand": None,
                "cat_no": [],
                "manufacturer_link": None,
                "material": None,
                "notes": None,
                "plr_directory": os.path.dirname(file_path),
            }
            
            # Extract manufacturer from the first subdirectory after "resources"
            path_parts = os.path.normpath(file_path).split(os.path.sep)
            if "resources" in path_parts:
                res_index = path_parts.index("resources")
                if res_index + 1 < len(path_parts):
                    manufacturer = path_parts[res_index + 1]
                    func_data["manufacturer"] = manufacturer.capitalize()
            
            # Check for return statement
            for stmt in node.body:
                if isinstance(stmt, ast.Return) and isinstance(stmt.value, ast.Call):
                    if isinstance(stmt.value.func, ast.Name) and stmt.value.func.id in imported_classes:
                        func_data["resource_type"] = stmt.value.func.id
            
            # Check for catalog numbers, material, manufacturer link, notes, and optional attributes in the docstring
            if node.body and isinstance(node.body[0], ast.Expr) and isinstance(node.body[0].value, ast.Str):
                docstring = node.body[0].value.s
                
                # Extract catalog numbers (handles both "cat. no." and "cat. no.s:")
                cat_match = re.search(r'cat\. no\.?s?:\s*([\w-]+(?:,\s*[\w-]+)*)', docstring, re.IGNORECASE)
                if cat_match:
                    func_data["cat_no"] = [num.strip() for num in cat_match.group(1).split(",")]
                
                # Extract manufacturer_link (ensures only the correct link is captured)
                link_match = re.search(r'manufacturer_link:\s*(https?://\S+)', docstring, re.IGNORECASE)
                if link_match:
                    link_start = link_match.start(1)
                    link_lines = []
                    for line in docstring[link_start:].split("\n"):
                        stripped_line = line.strip()
                        if stripped_line.startswith("-"):
                            break
                        link_lines.append(stripped_line)
                    func_data["manufacturer_link"] = "".join(link_lines)
                
                # Extract notes
                notes_match = re.search(r'notes:\s*(.*)', docstring, re.IGNORECASE)
                if notes_match:
                    func_data["notes"] = notes_match.group(1).strip()
                
                # Extract optional attributes
                for attr in optional_attr:
                    attr_match = re.search(fr'{attr}:\s*(.*)', docstring, re.IGNORECASE)
                    if attr_match:
                        func_data[attr] = attr_match.group(1).strip()
            
            if func_data["resource_type"]:
                function_map.append(func_data)
    
    return function_map


# Example usage:
class_list = ["Resource", "Carrier", "MFXCarrier", "PlateCarrier", "TipCarrier", "TroughCarrier", "TubeCarrier",
              "Container", "PetriDish", "Trough", "Tube", "Well", "Deck", "OTDeck", "HamiltonDeck", "TecanDeck",
              "ItemizedResource", "Plate", "TipRack", "TubeRack", "ResourceHolder", "PlateHolder", "Lid", 
              "PlateAdapter", "ResourceStack", "Tip"]

ignore_list = [
    "__init__"
]

all_py_files_dict = find_python_dirs("pylabrobot/resources/", ignore_list=ignore_list)

In [7]:
%%time

plr_rl_summary_dict = []
for py_file in all_py_files_dict:
    plr_rl_summary_dict.extend(
        extract_functions_with_imports(py_file, class_list, optional_attr=[
            "brand",
            "material",
            "tc_treated",
            "tech_drawing",
            "distributor"
        ])
    )

CPU times: user 94.2 ms, sys: 31.2 ms, total: 125 ms
Wall time: 399 ms


In [8]:
import pandas as pd

df = pd.DataFrame(plr_rl_summary_dict)
df

,plr_id,resource_type,manufacturer,brand,cat_no,manufacturer_link,material,notes,plr_directory,tech_drawing,tc_treated,distributor
0,AGenBio_4_wellplate_Vb,Plate,Agenbio,None,[],None,Polypropylene,None,pylabrobot/resources/agenbio,NaN,NaN,NaN
1,AGenBio_1_troughplate_190000uL_Fl,Plate,Agenbio,None,[],None,Polypropylene,None,pylabrobot/resources/agenbio,NaN,NaN,NaN
2,AGenBio_1_troughplate_100000uL_Fl,Plate,Agenbio,None,[],None,Polypropylene,None,pylabrobot/resources/agenbio,NaN,NaN,NaN
3,Alpaqua_96_magnum_flx,PlateAdapter,Alpaqua,None,[A000400],None,None,None,pylabrobot/resources/alpaqua,NaN,NaN,NaN
4,Azenta4titudeFrameStar_96_wellplate_200ul_Vb,Plate,Azenta,None,[4ti-0960],None,"Polypropylene wells, polycarbonate frame",None,pylabrobot/resources/azenta,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
169,Thermo_TS_96_wellplate_1200ul_Rb,Plate,Thermo_fisher,Thermo Scientific.,[AB1127],None,"Polypropylene (AB-1068, polystyrene).",None,pylabrobot/resources/thermo_fisher,NaN,NaN,NaN
170,Thermo_AB_96_wellplate_300ul_Vb_EnduraPlate,Plate,Thermo_fisher,None,[4483354],None,"Polycarbonate, Polypropylene.",None,pylabrobot/resources/thermo_fisher,NaN,NaN,NaN
171,Thermo_Nunc_96_well_plate_1300uL_Rb,Plate,Thermo_fisher,None,[],None,None,None,pylabrobot/resources/thermo_fisher,NaN,NaN,NaN
172,ThermoFisherMatrixTrough8094,Trough,Thermo_fisher,None,[],None,None,None,pylabrobot/resources/thermo_fisher,NaN,NaN,NaN


In [9]:
df.loc[df.manufacturer == "Corning"]#.manufacturer_link.iloc[0]

,plr_id,resource_type,manufacturer,brand,cat_no,manufacturer_link,material,notes,plr_directory,tech_drawing,tc_treated,distributor
18,Cor_Axy_24_wellplate_10mL_Vb,Plate,Corning,Axygen,[P-DW-10ML-24-C],https://ecatalog.corning.com/life-sciences/b2b...,Polypropylene,None,pylabrobot/resources/corning/axygen,NaN,NaN,NaN
19,Cor_Cos_6_wellplate_16800ul_Fb_Lid,Lid,Corning,Costar,[],None,None,None,pylabrobot/resources/corning/costar,NaN,NaN,NaN
20,Cor_Cos_6_wellplate_16800ul_Fb,Plate,Corning,Costar,"[3335, 3506, 3516, 3471]",https://ecatalog.corning.com/life-sciences/b2b...,Polypropylene,None,pylabrobot/resources/corning/costar,tech_drawings/Cor_Cos_6_wellplate_16800ul_Fb.pdf,NaN,NaN
21,Cor_12_wellplate_6900ul_Fb,Plate,Corning,None,"[3336, 3512, 3513]",None,None,None,pylabrobot/resources/corning/costar,NaN,NaN,NaN
22,Cor_24_wellplate_3470ul_Fb,Plate,Corning,None,"[3337, 3524, 3526, 3527, 3473]",None,None,None,pylabrobot/resources/corning/costar,NaN,NaN,NaN
23,Cor_48_wellplate_1620ul_Fb,Plate,Corning,None,[3548],None,None,None,pylabrobot/resources/corning/costar,NaN,NaN,NaN
24,Cos_96_wellplate_2mL_Vb,Plate,Corning,None,[3960],None,Polypropylene,None,pylabrobot/resources/corning/costar,NaN,NaN,NaN
25,Cor_Falcon_96_wellplate_275ul_Fb,Plate,Corning,Falcon,[353072],https://ecatalog.corning.com/life-sciences/b2b...,Polypropylene,None,pylabrobot/resources/corning/falcon,NaN,NaN,NaN
26,Cor_Falcon_96_wellplate_250ul_Rb,Plate,Corning,Falcon,[353077],https://ecatalog.corning.com/life-sciences/b2b...,Polystyrene,None,pylabrobot/resources/corning/falcon,https://www.corning.com/catalog/cls/documents/...,Yes,NaN
27,Cor_Falcon_96_wellplate_340ul_Fl_Black,Plate,Corning,Falcon,[353219],https://ecatalog.corning.com/life-sciences/b2b...,Polystyrene,None,pylabrobot/resources/corning/falcon,https://www.corning.com/catalog/cls/documents/...,Yes,NaN
